In [1]:
import pandas as pd
import numpy as np
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import psycopg2
from sqlalchemy import create_engine
import os
import plotly.graph_objects as go
import plotly.express as px
from bson.json_util import loads, dumps

In [2]:
demo = pd.read_csv('demo.csv')
food_brand = pd.read_csv('food_brand.csv')
food = pd.read_csv('food.csv')
pop_nutr = pd.read_csv('pop_nutr.csv')
rec = pd.read_csv('rec.csv')
units = pd.read_csv('units.csv')

In [3]:
demo = demo.rename({'SEQN':'seqn'},
                            axis=1)
demo

,seqn,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,College graduate or above,Never married,5.00
4,109269.0,M,2.0,Other Hispanic,NaN,NaN,0.96
...,...,...,...,...,...,...,...
11722,112157.0,M,3.0,Mexican American,NaN,NaN,0.48
11723,115293.0,F,2.0,Non-Hispanic White,NaN,NaN,0.99
11724,117889.0,F,2.0,Other Hispanic,NaN,NaN,0.85
11725,122957.0,F,2.0,Non-Hispanic Black,NaN,NaN,1.47


In [4]:
pop_nutr = pop_nutr.rename({'SEQN':'seqn'},
                            axis=1)
pop_nutr

,seqn,avg_vit_e,avg_vit_a,avg_thiamin_b1,avg_riboflavin_b2,avg_niacin,avg_vit_b6,avg_folate,avg_choline,avg_vit_b12,avg_vit_c,avg_vit_d,avg_vit_k,avg_calcium,avg_phosphorus,avg_magnesium,avg_iron,avg_zinc,avg_sodium,avg_potassium
0,109263.0,4.645,304.5,1.6960,1.0150,15.4385,1.1695,422.0,139.15,2.345,167.45,230.0,35.30,1026.5,758.5,160.0,10.615,6.395,1923.0,1561.5
1,109264.0,5.100,1122.5,2.2000,1.8770,36.1480,3.5925,659.0,198.15,8.880,44.90,146.0,118.75,571.5,1055.5,196.5,28.245,16.430,2195.0,1381.0
2,109265.0,4.440,633.0,1.1845,1.5540,16.4525,1.4330,239.0,197.65,4.310,66.25,244.0,57.50,1112.0,1192.0,196.0,7.230,7.320,1991.0,2279.0
3,109266.0,11.050,420.0,1.2930,1.6440,12.5290,1.3310,282.0,223.15,2.430,79.50,162.0,91.20,1048.0,1246.5,342.0,11.550,7.635,2720.0,2404.0
4,109269.0,4.075,280.0,0.5835,0.8115,5.4995,0.3315,157.0,268.25,1.260,128.05,106.0,55.25,358.5,548.0,75.5,5.860,3.040,1503.5,1081.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11722,112157.0,6.410,555.0,2.7330,2.7110,43.5520,4.3110,560.0,215.50,9.870,43.80,328.0,38.20,785.0,1006.0,250.0,29.710,8.140,2726.0,2179.0
11723,115293.0,4.320,410.0,1.1100,2.1990,13.5170,0.8230,236.0,222.30,5.420,14.60,600.0,51.30,1459.0,1343.0,198.0,5.940,6.550,1677.0,2458.0
11724,117889.0,4.460,463.0,1.0850,1.4080,13.1980,1.1280,257.0,232.00,2.760,64.80,152.0,30.20,897.0,938.0,170.0,10.680,7.550,1721.0,1469.0
11725,122957.0,4.170,266.0,0.9370,1.1750,13.4810,1.1240,163.0,156.80,2.310,29.40,140.0,25.60,792.0,1030.0,201.0,7.260,6.240,2009.0,1718.0


In [5]:
food = food.rename({'fdcId':'fdcid',
                    'riboflavin':'riboflavin_b2',
                    'thiamin':'thiamin_b1'},
                            axis=1)
food

,fdcid,description,calcium,choline,folate,iron,magnesium,niacin,phosphorus,potassium,...,sodium,thiamin_b1,vit_a,vit_b12,vit_b6,vit_c,vit_d,vit_e,zinc,vit_k
0,2262074,"Almond butter, creamy",264.00,0.0,42.50,4.1100,268.00,3.9100,507.00,745.0,...,0.996,0.09460,0.0,0.000,0.0838,0.0,0.0,16.70,3.18000,0.0
1,2257045,"Almond milk, unsweetened, plain, refrigerated",158.00,0.0,0.00,0.1250,8.25,0.0888,19.10,49.3,...,59.200,0.00500,0.0,0.449,0.0000,0.0,63.6,0.00,0.07660,0.0
2,1999631,"Almond milk, unsweetened, plain, shelf stable",173.00,0.0,0.00,0.2900,6.78,0.0744,29.80,30.8,...,59.600,0.00000,41.4,0.338,0.0000,0.0,37.1,3.32,0.17200,0.0
3,2003590,"Apple juice, with added vitamin C, from concen...",7.10,0.0,3.09,0.0402,4.86,0.0103,5.99,95.9,...,4.870,0.00000,0.0,0.000,0.0139,51.2,0.0,0.00,0.00212,0.0
4,1750340,"Apples, fuji, with skin, raw",5.98,0.0,0.00,0.0150,4.67,0.0906,9.78,104.0,...,1.010,0.00625,0.0,0.000,0.0354,0.0,0.0,0.00,0.01680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13622,172219,"Yogurt, vanilla, low fat, fortified with vitam...",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,...,66.000,0.04200,12.0,0.530,0.0450,0.8,47.0,0.02,0.83000,0.1
13623,170888,"Yogurt, vanilla, low fat.",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,...,66.000,0.04200,12.0,0.530,0.0450,0.8,1.0,0.02,0.83000,0.1
13624,170906,"Yogurt, vanilla, non-fat",118.00,15.6,11.00,0.0000,16.00,0.1070,88.00,141.0,...,47.000,0.04200,61.0,0.530,0.0450,0.0,35.0,0.01,0.83000,0.0
13625,173730,"Yokan, prepared from adzuki beans and sugar",27.00,0.0,8.00,1.1600,18.00,0.0570,40.00,45.0,...,83.000,0.00500,0.0,0.000,0.0080,0.0,0.0,0.00,0.07000,0.0


In [6]:
os.chdir('..')
postgres_password = os.environ['POSTGRES_PASSWORD']
os.chdir('app')

In [7]:
server = psycopg2.connect(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432')
server.autocommit = True


In [8]:
cursor = server.cursor()

In [9]:
try:
    cursor.execute("CREATE DATABASE nutrecdb")
except:
    cursor.execute("DROP DATABASE nutrecdb")
    cursor.execute("CREATE DATABASE nutrecdb")

In [10]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432',
    db = 'nutrecdb'))

In [11]:
demo.to_sql('demo', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
food_brand.to_sql('food_brand', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
food.to_sql('food', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
pop_nutr.to_sql('pop_nutr', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
rec.to_sql('rec', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
units.to_sql('units', con = engine, if_exists = 'replace', index=False, chunksize = 1000)

19

In [12]:
rec

,age_lower,age_upper,sex,vit_a,vit_e,vit_d,vit_c,thiamin_b1,riboflavin_b2,niacin,...,choline,vit_k,folate,calcium,iron,magnesium,phosphorus,potassium,sodium,zinc
0,2,3,M/F,300,6,600,15,0.5,0.5,6,...,200,30,150,700,7,80,460,2000,1200,3
1,4,8,F,400,7,600,25,0.6,0.6,8,...,250,55,200,1000,10,130,500,2300,1500,5
2,9,13,F,600,11,600,45,0.9,0.9,12,...,375,60,300,1300,8,240,1250,2300,1800,8
3,14,18,F,700,15,600,65,1.0,1.0,14,...,400,75,400,1300,15,360,1250,2300,2300,9
4,19,30,F,700,15,600,75,1.1,1.1,14,...,425,90,400,1000,18,310,700,2600,2300,8
5,31,50,F,700,15,600,75,1.1,1.1,14,...,425,90,400,1000,18,320,700,2600,2300,8
6,51,150,F,700,15,600,75,1.1,1.1,14,...,425,90,400,1200,8,320,700,2600,2300,8
7,4,8,M,400,7,600,25,0.6,0.6,8,...,250,55,200,1000,10,130,500,2300,1500,5
8,9,13,M,600,11,600,45,0.9,0.9,12,...,375,60,300,1300,8,240,1250,2500,1800,8
9,14,18,M,900,15,600,75,1.2,1.3,16,...,550,75,400,1300,11,410,1250,3000,2300,11


In [13]:
#search_string = get_search_string(22, 'M', race = None, education = None, marital = None, poverty = 'Yes')
myquery = f'''
SELECT COUNT(d.age_year) as n, AVG(avg_vit_e) AS vit_e, AVG(avg_vit_a) AS vit_a, AVG(avg_thiamin_b1) AS thiamin_b1, AVG(avg_riboflavin_b2) AS riboflavin_b2, 
    AVG(avg_niacin) AS niacin, AVG(avg_vit_b6) AS vit_b6, AVG(avg_folate) AS folate, AVG(avg_choline) AS choline, AVG(avg_vit_b12) AS vit_b12, 
    AVG(avg_vit_c) AS vit_c, AVG(avg_vit_d) AS vit_d, AVG(avg_vit_k) AS vit_k, AVG(avg_calcium) AS calcium, AVG(avg_phosphorus) AS phosphorus, 
    AVG(avg_magnesium) AS magnesium, AVG(avg_iron) AS iron, AVG(avg_zinc) AS zinc, AVG(avg_sodium) AS sodium, AVG(avg_potassium) AS potassium
FROM demo d 
INNER JOIN pop_nutr p
    ON d.seqn = p.seqn
WHERE sex = 'F'
GROUP BY sex
'''

average_nutr = pd.read_sql(myquery, con=engine)
average_nutr

,n,vit_e,vit_a,thiamin_b1,riboflavin_b2,niacin,vit_b6,folate,choline,vit_b12,vit_c,vit_d,vit_k,calcium,phosphorus,magnesium,iron,zinc,sodium,potassium
0,5911,7.570145,530.449332,1.29896,1.575689,19.480873,1.574077,305.617916,258.35022,3.623612,72.196227,158.354593,102.402199,800.969379,1102.000677,238.088987,11.428141,8.294621,2751.129166,2084.55862


In [14]:
myquery = '''
SELECT vit_e, vit_a, thiamin_b1, riboflavin_b2, niacin, vit_b6, folate, choline, vit_b12, vit_c, vit_d, vit_k, calcium, phosphorus, 
    magnesium, iron, zinc, sodium, potassium
FROM rec
WHERE age_lower= 19 AND sex = 'F'
'''

recommendation = pd.read_sql(myquery, con=engine)
recommendation

,vit_e,vit_a,thiamin_b1,riboflavin_b2,niacin,vit_b6,folate,choline,vit_b12,vit_c,vit_d,vit_k,calcium,phosphorus,magnesium,iron,zinc,sodium,potassium
0,15,700,1.1,1.1,14,1.3,400,425,2.4,75,600,90,1000,700,310,18,8,2300,2600


In [15]:
needs = recommendation.subtract(average_nutr.drop(['n'], axis = 1))
vit_e = needs['vit_e'][0]
vit_a = needs['vit_a'][0]
thiamin_b1 = needs['thiamin_b1'][0]
riboflavin_b2 = needs['riboflavin_b2'][0]
niacin = needs['niacin'][0]
vit_b6 = needs['vit_b6'][0]
folate = needs['folate'][0]
choline = needs['choline'][0]
vit_b12 = needs['vit_b12'][0]
vit_c = needs['vit_c'][0]
vit_d = needs['vit_d'][0]
vit_k = needs['vit_k'][0]
calcium = needs['calcium'][0]
phosphorus = needs['phosphorus'][0]
magnesium = needs['magnesium'][0]
iron = needs['iron'][0]
zinc = needs['zinc'][0]
sodium = needs['sodium'][0]
potassium = needs['potassium'][0]

In [16]:
needs

,vit_e,vit_a,thiamin_b1,riboflavin_b2,niacin,vit_b6,folate,choline,vit_b12,vit_c,vit_d,vit_k,calcium,phosphorus,magnesium,iron,zinc,sodium,potassium
0,7.429855,169.550668,-0.19896,-0.475689,-5.480873,-0.274077,94.382084,166.64978,-1.223612,2.803773,441.645407,-12.402199,199.030621,-402.000677,71.911013,6.571859,-0.294621,-451.129166,515.44138


In [17]:
vit_e

7.429854508543403

In [18]:
multiplier = .1
myquery = f'''
WITH errors AS (SELECT fdcid, description,
    vit_e*{multiplier} AS vit_e,
    vit_a*{multiplier} AS vit_a,
    thiamin_b1*{multiplier} AS thiamin_b1,
    riboflavin_b2*{multiplier} AS riboflavin_b2,
    niacin*{multiplier} AS niacin,
    vit_b6*{multiplier} AS vit_b6,
    folate*{multiplier} AS folate,
    choline*{multiplier} AS choline,
    vit_b12*{multiplier} AS vit_b12,
    vit_c*{multiplier} AS vit_c,
    vit_d*{multiplier} AS vit_d,
    vit_k*{multiplier} AS vit_k,
    calcium*{multiplier} AS calcium,
    phosphorus*{multiplier} AS phosphorus,
    magnesium*{multiplier} AS magnesium,
    iron*{multiplier} AS iron,
    zinc*{multiplier} AS zinc,
    sodium*{multiplier} AS sodium,
    potassium*{multiplier} AS potassium,
    POWER(({multiplier}*vit_e - {vit_e}), 2) as vit_e_2, 
    POWER(({multiplier}*vit_a - {vit_a}), 2) as vit_a_2, 
    POWER(({multiplier}*thiamin_b1 - {thiamin_b1}), 2) as thiamin_b1_2, 
    POWER(({multiplier}*riboflavin_b2 - {riboflavin_b2}), 2) as riboflavin_b2_2, 
    POWER(({multiplier}*niacin - {niacin}), 2) as niacin_2, 
    POWER(({multiplier}*vit_b6 - {vit_b6}), 2) as vit_b6_2, 
    POWER(({multiplier}*folate - {folate}), 2) as folate_2, 
    POWER(({multiplier}*choline - {choline}), 2) as choline_2, 
    POWER(({multiplier}*vit_b12 - {vit_b12}), 2) as vit_b12_2, 
    POWER(({multiplier}*vit_c - {vit_c}), 2) as vit_c_2, 
    POWER(({multiplier}*vit_d - {vit_d}), 2) as vit_d_2, 
    POWER(({multiplier}*vit_k - {vit_k}), 2) as vit_k_2, 
    POWER(({multiplier}*calcium - {calcium}), 2) as calcium_2, 
    POWER(({multiplier}*phosphorus - {phosphorus}), 2) as phosphorus_2, 
    POWER(({multiplier}*magnesium - {magnesium}), 2) as magnesium_2, 
    POWER(({multiplier}*iron - {iron}), 2) as iron_2, 
    POWER(({multiplier}*zinc - {zinc}), 2) as zinc_2, 
    POWER(({multiplier}*sodium - {sodium}), 2) as sodium_2, 
    POWER(({multiplier}*potassium - {potassium}), 2) as potassium_2,
    {multiplier}*100.0 as serving
FROM food)

SELECT fdcid, description, serving, sqrt((vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
    vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2)/19) AS rmse,
    vit_e,
    vit_a,
    thiamin_b1,
    riboflavin_b2,
    niacin,
    vit_b6,
    folate,
    choline,
    vit_b12,
    vit_c,
    vit_d,
    vit_k,
    calcium,
    phosphorus,
    magnesium,
    iron,
    zinc,
    sodium,
    potassium
FROM errors
ORDER BY rmse ASC
LIMIT 100
'''

foods = pd.read_sql(myquery, con=engine)

multipliers = np.arange(.2, 4, 0.1).tolist()
for multiplier in multipliers:
    myquery = f'''
    WITH errors AS (SELECT fdcid, description,
        vit_e*{multiplier} AS vit_e,
        vit_a*{multiplier} AS vit_a,
        thiamin_b1*{multiplier} AS thiamin_b1,
        riboflavin_b2*{multiplier} AS riboflavin_b2,
        niacin*{multiplier} AS niacin,
        vit_b6*{multiplier} AS vit_b6,
        folate*{multiplier} AS folate,
        choline*{multiplier} AS choline,
        vit_b12*{multiplier} AS vit_b12,
        vit_c*{multiplier} AS vit_c,
        vit_d*{multiplier} AS vit_d,
        vit_k*{multiplier} AS vit_k,
        calcium*{multiplier} AS calcium,
        phosphorus*{multiplier} AS phosphorus,
        magnesium*{multiplier} AS magnesium,
        iron*{multiplier} AS iron,
        zinc*{multiplier} AS zinc,
        sodium*{multiplier} AS sodium,
        potassium*{multiplier} AS potassium,
        POWER(({multiplier}*vit_e - {vit_e}), 2) as vit_e_2, 
        POWER(({multiplier}*vit_a - {vit_a}), 2) as vit_a_2, 
        POWER(({multiplier}*thiamin_b1 - {thiamin_b1}), 2) as thiamin_b1_2, 
        POWER(({multiplier}*riboflavin_b2 - {riboflavin_b2}), 2) as riboflavin_b2_2, 
        POWER(({multiplier}*niacin - {niacin}), 2) as niacin_2, 
        POWER(({multiplier}*vit_b6 - {vit_b6}), 2) as vit_b6_2, 
        POWER(({multiplier}*folate - {folate}), 2) as folate_2, 
        POWER(({multiplier}*choline - {choline}), 2) as choline_2, 
        POWER(({multiplier}*vit_b12 - {vit_b12}), 2) as vit_b12_2, 
        POWER(({multiplier}*vit_c - {vit_c}), 2) as vit_c_2, 
        POWER(({multiplier}*vit_d - {vit_d}), 2) as vit_d_2, 
        POWER(({multiplier}*vit_k - {vit_k}), 2) as vit_k_2, 
        POWER(({multiplier}*calcium - {calcium}), 2) as calcium_2, 
        POWER(({multiplier}*phosphorus - {phosphorus}), 2) as phosphorus_2, 
        POWER(({multiplier}*magnesium - {magnesium}), 2) as magnesium_2, 
        POWER(({multiplier}*iron - {iron}), 2) as iron_2, 
        POWER(({multiplier}*zinc - {zinc}), 2) as zinc_2, 
        POWER(({multiplier}*sodium - {sodium}), 2) as sodium_2, 
        POWER(({multiplier}*potassium - {potassium}), 2) as potassium_2,
        {multiplier}*100.0 as serving
    FROM food)

    SELECT fdcid, description, serving, SQRT((vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
        vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2)/19) AS rmse,
        vit_e,
        vit_a,
        thiamin_b1,
        riboflavin_b2,
        niacin,
        vit_b6,
        folate,
        choline,
        vit_b12,
        vit_c,
        vit_d,
        vit_k,
        calcium,
        phosphorus,
        magnesium,
        iron,
        zinc,
        sodium,
        potassium
    FROM errors
    ORDER BY rmse ASC
    LIMIT 100
    '''

    new_foods = pd.read_sql(myquery, con=engine)
    foods = pd.concat([foods, new_foods], axis=0)

In [19]:
pd.set_option('display.max_rows', 50)
foods.sort_values(by='rmse', ascending=True).drop_duplicates(subset=['description'])[0:50]

,fdcid,description,serving,rmse,vit_e,vit_a,thiamin_b1,riboflavin_b2,niacin,vit_b6,...,vit_c,vit_d,vit_k,calcium,phosphorus,magnesium,iron,zinc,sodium,potassium
0,168422,"Mushrooms, Chanterelle, raw",110.0,174.056685,0.000,0.0,0.0165,0.2365,4.4880,0.0484,...,0.00,233.2,0.00,16.5,62.7,14.3,3.817,0.781,9.9,556.6
0,167794,"Orange juice, chilled, includes from concentra...",220.0,176.692357,2.486,250.8,0.1012,0.0858,0.6160,0.1672,...,73.92,92.4,0.00,321.2,37.4,24.2,0.286,0.154,4.4,391.6
0,169377,"Mushrooms, portabella, exposed to ultraviolet ...",80.0,177.314763,0.000,0.0,0.0576,0.3224,5.0080,0.0976,...,0.00,419.2,5.84,2.4,108.0,10.4,0.320,0.520,8.8,349.6
1,168488,"Taro leaves, cooked, steamed, without salt",110.0,181.861293,0.000,233.2,0.1529,0.4180,1.3970,0.0792,...,39.05,0.0,0.00,94.6,29.7,22.0,1.298,0.231,2.2,506.0
0,168557,"Mushrooms, brown, italian, or crimini, exposed...",40.0,181.882413,0.004,0.0,0.0380,0.1960,1.5200,0.0440,...,0.00,512.0,0.00,7.2,48.0,3.6,0.160,0.440,2.4,179.2
0,169815,"Lambsquarters, steamed (Northern Plains Indians)",50.0,181.888945,0.000,97.0,0.0235,0.1350,0.3115,0.1160,...,2.45,0.0,0.00,174.5,28.0,82.0,0.575,0.305,2.0,535.0
0,173525,"Infant formula, MEAD JOHNSON, Gentlease, ready...",390.0,182.222215,2.301,230.1,0.2028,0.3588,2.5740,0.1521,...,30.81,152.1,23.01,210.6,132.6,19.5,4.602,2.574,0.0,327.6
3,169224,"Cowpeas, leafy tips, raw",110.0,183.455953,0.000,39.6,0.3894,0.1925,1.2320,0.1947,...,39.60,0.0,0.00,69.3,9.9,47.3,2.112,0.319,7.7,500.5
1,169376,"Mushroom, white, exposed to ultraviolet light,...",50.0,183.929806,0.005,0.0,0.0405,0.2010,1.8050,0.0520,...,1.05,525.0,0.50,1.5,43.0,4.5,0.250,0.260,2.5,159.0
3,170143,"Mushrooms, portabella, exposed to ultraviolet ...",50.0,184.475266,0.010,0.0,0.0295,0.0650,2.2450,0.0740,...,0.00,570.0,0.90,1.5,54.0,5.0,0.155,0.265,4.5,182.0


In [20]:
def foodgraph(df):
    #df = members.query(f"propublica_id == '{propub}'")
    recommendation = df.sort_values(by='rmse', ascending=True).drop_duplicates(subset=['description'])[0:100]
    fig = px.scatter(recommendation, x = 'serving', y = 'rmse', labels = {'serving':'Serving (grams)',
                                                                 'rmse': 'Deficit Fulfillment Score (Lower is Better)'},
                    height = 600, width = 600,
                    opacity = .5,
                    hover_data = ['description'])

    #fig.add_trace(go.Scatter(x = df['DWNOMINATE'], y = df['votes_with_party_pct'], marker = dict(size = 12),
                             #marker_symbol = 'star'))
    return fig
#foodgraph()

In [21]:
multiplier = .1
myquery = f'''
WITH errors AS (SELECT fdcid, description, 
    ABS(({multiplier}*vit_e - {vit_e})) as vit_e_2, 
    ABS(({multiplier}*vit_a - {vit_a})) as vit_a_2, 
    ABS(({multiplier}*thiamin_b1 - {thiamin_b1})) as thiamin_b1_2, 
    ABS(({multiplier}*riboflavin_b2 - {riboflavin_b2})) as riboflavin_b2_2, 
    ABS(({multiplier}*niacin - {niacin})) as niacin_2, 
    ABS(({multiplier}*vit_b6 - {vit_b6})) as vit_b6_2, 
    ABS(({multiplier}*folate - {folate})) as folate_2, 
    ABS(({multiplier}*choline - {choline})) as choline_2, 
    ABS(({multiplier}*vit_b12 - {vit_b12})) as vit_b12_2, 
    ABS(({multiplier}*vit_c - {vit_c})) as vit_c_2, 
    ABS(({multiplier}*vit_d - {vit_d})) as vit_d_2, 
    ABS(({multiplier}*vit_k - {vit_k})) as vit_k_2, 
    ABS(({multiplier}*calcium - {calcium})) as calcium_2, 
    ABS(({multiplier}*phosphorus - {phosphorus})) as phosphorus_2, 
    ABS(({multiplier}*magnesium - {magnesium})) as magnesium_2, 
    ABS(({multiplier}*iron - {iron})) as iron_2, 
    ABS(({multiplier}*zinc - {zinc})) as zinc_2, 
    ABS(({multiplier}*sodium - {sodium})) as sodium_2, 
    ABS(({multiplier}*potassium - {potassium})) as potassium_2
FROM food)

SELECT fdcid, description, vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
    vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2 AS sum_squares
FROM errors
ORDER BY sum_squares ASC
LIMIT 20
'''

foods = pd.read_sql(myquery, con=engine)

multipliers = np.arange(.2, 5, 0.1).tolist()
for multiplier in multipliers:
    myquery = f'''
    WITH errors AS (SELECT fdcid, description, 
        ABS({multiplier}*vit_e - {vit_e}) as vit_e_2, 
        ABS(({multiplier}*vit_a - {vit_a})) as vit_a_2, 
        ABS(({multiplier}*thiamin_b1 - {thiamin_b1})) as thiamin_b1_2, 
        ABS(({multiplier}*riboflavin_b2 - {riboflavin_b2})) as riboflavin_b2_2, 
        ABS(({multiplier}*niacin - {niacin})) as niacin_2, 
        ABS(({multiplier}*vit_b6 - {vit_b6})) as vit_b6_2, 
        ABS(({multiplier}*folate - {folate})) as folate_2, 
        ABS(({multiplier}*choline - {choline})) as choline_2, 
        ABS(({multiplier}*vit_b12 - {vit_b12})) as vit_b12_2, 
        ABS(({multiplier}*vit_c - {vit_c})) as vit_c_2, 
        ABS(({multiplier}*vit_d - {vit_d})) as vit_d_2, 
        ABS(({multiplier}*vit_k - {vit_k})) as vit_k_2, 
        ABS(({multiplier}*calcium - {calcium})) as calcium_2, 
        ABS(({multiplier}*phosphorus - {phosphorus})) as phosphorus_2, 
        ABS(({multiplier}*magnesium - {magnesium})) as magnesium_2, 
        ABS(({multiplier}*iron - {iron})) as iron_2, 
        ABS(({multiplier}*zinc - {zinc})) as zinc_2, 
        ABS(({multiplier}*sodium - {sodium})) as sodium_2, 
        ABS(({multiplier}*potassium - {potassium})) as potassium_2
    FROM food)

    SELECT fdcid, description, vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
        vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2 AS sum_squares
    FROM errors
    WHERE description NOT LIKE '%%JUICE%%' AND description NOT LIKE '%%DURUM%%'
    ORDER BY sum_squares ASC
    LIMIT 20
    '''

    new_foods = pd.read_sql(myquery, con=engine)
    foods = pd.concat([foods, new_foods], axis=0)

In [22]:
pd.set_option('display.max_rows', 30)
foods.sort_values(by='sum_squares', ascending=True).drop_duplicates(subset=['description'])[0:30]

,fdcid,description,sum_squares
0,173474,"Peppermint, fresh",1666.155295
0,173475,"Spearmint, fresh",1707.928545
1,173473,"Rosemary, fresh",1714.935539
0,175136,"Fish, salmon, coho, wild, raw",1720.202714
1,171994,"Fish, mackerel, Pacific and jack, mixed specie...",1721.228739
1,173672,"Fish, mackerel, Pacific and jack, mixed specie...",1721.246714
0,168437,"Mushrooms, shiitake, cooked, without salt",1737.782714
2,168419,"Jute, potherb, raw",1742.335364
0,169815,"Lambsquarters, steamed (Northern Plains Indians)",1747.557526
1,173698,"Fish, snapper, mixed species, raw",1762.903800


In [23]:
len(multipliers)

48

In [24]:
myquery = f'''
SELECT fdcid, description, vit_e, vit_a, thiamin_b1, riboflavin_b2, niacin, vit_b6, folate, choline, vit_b12, vit_c, vit_d, vit_k, calcium, phosphorus, 
    magnesium, iron, zinc, sodium, potassium
FROM food
'''

foods = pd.read_sql(myquery, con=engine)
foods

,fdcid,description,vit_e,vit_a,thiamin_b1,riboflavin_b2,niacin,vit_b6,folate,choline,...,vit_c,vit_d,vit_k,calcium,phosphorus,magnesium,iron,zinc,sodium,potassium
0,2262074,"Almond butter, creamy",16.70,0.0,0.09460,0.0000,3.9100,0.0838,42.50,0.0,...,0.0,0.0,0.0,264.00,507.00,268.00,4.1100,3.18000,0.996,745.0
1,2257045,"Almond milk, unsweetened, plain, refrigerated",0.00,0.0,0.00500,0.0833,0.0888,0.0000,0.00,0.0,...,0.0,63.6,0.0,158.00,19.10,8.25,0.1250,0.07660,59.200,49.3
2,1999631,"Almond milk, unsweetened, plain, shelf stable",3.32,41.4,0.00000,0.0334,0.0744,0.0000,0.00,0.0,...,0.0,37.1,0.0,173.00,29.80,6.78,0.2900,0.17200,59.600,30.8
3,2003590,"Apple juice, with added vitamin C, from concen...",0.00,0.0,0.00000,0.0000,0.0103,0.0139,3.09,0.0,...,51.2,0.0,0.0,7.10,5.99,4.86,0.0402,0.00212,4.870,95.9
4,1750340,"Apples, fuji, with skin, raw",0.00,0.0,0.00625,0.0675,0.0906,0.0354,0.00,0.0,...,0.0,0.0,0.0,5.98,9.78,4.67,0.0150,0.01680,1.010,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13622,172219,"Yogurt, vanilla, low fat, fortified with vitam...",0.02,12.0,0.04200,0.2010,0.1070,0.0450,11.00,15.2,...,0.8,47.0,0.1,171.00,135.00,16.00,0.0700,0.83000,66.000,219.0
13623,170888,"Yogurt, vanilla, low fat.",0.02,12.0,0.04200,0.2010,0.1070,0.0450,11.00,15.2,...,0.8,1.0,0.1,171.00,135.00,16.00,0.0700,0.83000,66.000,219.0
13624,170906,"Yogurt, vanilla, non-fat",0.01,61.0,0.04200,0.2000,0.1070,0.0450,11.00,15.6,...,0.0,35.0,0.0,118.00,88.00,16.00,0.0000,0.83000,47.000,141.0
13625,173730,"Yokan, prepared from adzuki beans and sugar",0.00,0.0,0.00500,0.0040,0.0570,0.0080,8.00,0.0,...,0.0,0.0,0.0,27.00,40.00,18.00,1.1600,0.07000,83.000,45.0


In [25]:
a = ['something']
b = None
if b:
    a.append('something else')
a

['something']

In [26]:
rec

,age_lower,age_upper,sex,vit_a,vit_e,vit_d,vit_c,thiamin_b1,riboflavin_b2,niacin,...,choline,vit_k,folate,calcium,iron,magnesium,phosphorus,potassium,sodium,zinc
0,2,3,M/F,300,6,600,15,0.5,0.5,6,...,200,30,150,700,7,80,460,2000,1200,3
1,4,8,F,400,7,600,25,0.6,0.6,8,...,250,55,200,1000,10,130,500,2300,1500,5
2,9,13,F,600,11,600,45,0.9,0.9,12,...,375,60,300,1300,8,240,1250,2300,1800,8
3,14,18,F,700,15,600,65,1.0,1.0,14,...,400,75,400,1300,15,360,1250,2300,2300,9
4,19,30,F,700,15,600,75,1.1,1.1,14,...,425,90,400,1000,18,310,700,2600,2300,8
5,31,50,F,700,15,600,75,1.1,1.1,14,...,425,90,400,1000,18,320,700,2600,2300,8
6,51,150,F,700,15,600,75,1.1,1.1,14,...,425,90,400,1200,8,320,700,2600,2300,8
7,4,8,M,400,7,600,25,0.6,0.6,8,...,250,55,200,1000,10,130,500,2300,1500,5
8,9,13,M,600,11,600,45,0.9,0.9,12,...,375,60,300,1300,8,240,1250,2500,1800,8
9,14,18,M,900,15,600,75,1.2,1.3,16,...,550,75,400,1300,11,410,1250,3000,2300,11


In [27]:
def get_search_string(age, sex, race = None, education = None, marital = None, poverty = None):
    search_string = ""
    search_string_rec = ""
    if age >= 2 and age <= 3:
        search_string = search_string + "d.age_year >= 2 AND d.age_year <= 3"
        search_string_rec = search_string_rec + "age_lower = 2 AND sex = 'M/F'"
    elif age >= 4 and age <= 8:
        search_string = search_string + "d.age_year >= 4 AND d.age_year <= 8"
        search_string_rec = search_string_rec + "age_lower = 4"
    elif age >= 9 and age <= 13:
        search_string = search_string + "d.age_year >= 9 AND d.age_year <= 13"
        search_string_rec = search_string_rec + "age_lower = 9"
    elif age >= 14 and age <= 18:
        search_string = search_string + "d.age_year >= 14 AND d.age_year <= 18"
        search_string_rec = search_string_rec + "age_lower = 14"
    elif age >= 19 and age <= 30:
        search_string = search_string + "d.age_year >= 19 AND d.age_year <= 30"
        search_string_rec = search_string_rec + "age_lower = 19"
    elif age >= 31 and age <= 50:
        search_string = search_string + "d.age_year >= 31 AND d.age_year <= 50"
        search_string_rec = search_string_rec + "age_lower = 31"
    else:
        search_string = search_string + "d.age_year >= 50"
        search_string_rec = search_string_rec + "age_lower = 51"
    if sex == 'F' and age > 3:
        search_string = search_string + " AND d.sex = 'F'"
        search_string_rec = search_string_rec + " AND sex = 'F'"
    elif sex == 'M' and age > 3:
        search_string = search_string + " AND d.sex = 'M'"
        search_string_rec = search_string_rec + " AND sex = 'M'"
    if race:
        if race == 'Non-Hispanic Asian':
            search_string = search_string + " AND d.race = 'Non-Hispanic Asian'"
        elif race == 'Mexican American':
            search_string = search_string + " AND d.race = 'Mexican American'"
        elif race == 'Non-Hispanic White':
            search_string = search_string + " AND d.race = 'Non-Hispanic White'"
        elif race == 'Other Hispanic':
            search_string = search_string + " AND d.race = 'Other Hispanic'"
        elif race =='Non-Hispanic Black':
            search_string = search_string + " AND d.race = 'Non-Hispanic Black'"
        else:
            search_string = search_string + " AND d.race = 'Other Race - Including Multi-Racial'"
    if education:
        if education == 'Less than 9th grade' and age > 19:
            search_string = search_string + " AND d.education = 'Less than 9th grade'"
        elif education == '9-11th grade' and age > 19:
            search_string = search_string + " AND d.education = '9-11th grade'"
        elif education == 'High school graduate/GED or equivalent' and age > 19:
            search_string = search_string + " AND d.education = 'High school graduate/GED or equivalent'"
        elif education == 'Some college or AA degree' and age > 19:
            search_string = search_string + " AND d.education = 'Some college or AA degree'"
        else:
            search_string = search_string + " AND d.education = 'College graduate or above'"
    if marital:
        if marital == 'Never married':
            search_string = search_string + " AND d.marital_status = 'Never married'"
        elif marital == 'Married/Living with Partner':
            search_string = search_string + " AND d.marital_status = 'Married/Living with Partner'"
        else:
            search_string = search_string + " AND d.marital_status = 'Widowed/Divorced/Separated'"
    if poverty:
        if poverty == 'Yes':
            search_string = search_string + " AND d.income_pov_rat <= 1"
        else: 
            search_string = search_string + " AND d.income_pov_rat > 1"
    return search_string, search_string_rec

In [28]:
get_search_string(22, 'M', race = 'Non-Hispanic White', education = None, marital = None, poverty = None)

("d.age_year >= 19 AND d.age_year <= 30 AND d.sex = 'M' AND d.race = 'Non-Hispanic White'",
 "age_lower = 19 AND sex = 'M'")

In [29]:
'd.age_year >= 2 AND d.age_year <= 3' + ' AND d.sex = M/F'

'd.age_year >= 2 AND d.age_year <= 3 AND d.sex = M/F'

In [30]:
demo['income_pov_rat']

0        4.66
1        0.83
2        3.06
3        5.00
4        0.96
         ... 
11722    0.48
11723    0.99
11724    0.85
11725    1.47
11726    5.00
Name: income_pov_rat, Length: 11727, dtype: float64

In [31]:
# how to get the graph up

def make_nutrition_graph(age, sex, race = None, education = None, marital = None, poverty = None):
    
    # get the string to search the demo and population nutrition tables
    
    strings = get_search_string(age, sex, race, education, marital, poverty)
    dem_string = strings[0]
    nutr_string = strings[1]

    # search the demo and population nutrition tables
    
    mean_nutr_query = f'''
    SELECT COUNT(d.age_year) as n, AVG(avg_vit_e) AS vit_e, AVG(avg_vit_a) AS vit_a, AVG(avg_thiamin_b1) AS thiamin_b1, AVG(avg_riboflavin_b2) AS riboflavin_b2, 
        AVG(avg_niacin) AS niacin, AVG(avg_vit_b6) AS vit_b6, AVG(avg_folate) AS folate, AVG(avg_choline) AS choline, AVG(avg_vit_b12) AS vit_b12, 
        AVG(avg_vit_c) AS vit_c, AVG(avg_vit_d) AS vit_d, AVG(avg_vit_k) AS vit_k, AVG(avg_calcium) AS calcium, AVG(avg_phosphorus) AS phosphorus, 
        AVG(avg_magnesium) AS magnesium, AVG(avg_iron) AS iron, AVG(avg_zinc) AS zinc, AVG(avg_sodium) AS sodium, AVG(avg_potassium) AS potassium
    FROM demo d 
    INNER JOIN pop_nutr p
        ON d.seqn = p.seqn
    WHERE {dem_string}
    GROUP BY sex
    '''

    mean_nutr = pd.read_sql(mean_nutr_query, con=engine)
    
    myquery = f'''
    SELECT vit_e, vit_a, thiamin_b1, riboflavin_b2, niacin, vit_b6, folate, choline, vit_b12, vit_c, vit_d, vit_k, calcium, phosphorus, 
        magnesium, iron, zinc, sodium, potassium
    FROM rec
    WHERE {nutr_string}
    '''

    recommendation = pd.read_sql(myquery, con=engine)

    # subtract the searches to get the deficit and store those deficit values
    
    needs = recommendation.subtract(mean_nutr.drop(['n'], axis = 1))
    vit_e = needs['vit_e'][0]
    vit_a = needs['vit_a'][0]
    thiamin_b1 = needs['thiamin_b1'][0]
    riboflavin_b2 = needs['riboflavin_b2'][0]
    niacin = needs['niacin'][0]
    vit_b6 = needs['vit_b6'][0]
    folate = needs['folate'][0]
    choline = needs['choline'][0]
    vit_b12 = needs['vit_b12'][0]
    vit_c = needs['vit_c'][0]
    vit_d = needs['vit_d'][0]
    vit_k = needs['vit_k'][0]
    calcium = needs['calcium'][0]
    phosphorus = needs['phosphorus'][0]
    magnesium = needs['magnesium'][0]
    iron = needs['iron'][0]
    zinc = needs['zinc'][0]
    sodium = needs['sodium'][0]
    potassium = needs['potassium'][0]

    # calculate least squares for foods and order them smallest to largest
    
    multiplier = .1
    myquery = f'''
    WITH errors AS (SELECT fdcid, description,
        vit_e*{multiplier} AS vit_e,
        vit_a*{multiplier} AS vit_a,
        thiamin_b1*{multiplier} AS thiamin_b1,
        riboflavin_b2*{multiplier} AS riboflavin_b2,
        niacin*{multiplier} AS niacin,
        vit_b6*{multiplier} AS vit_b6,
        folate*{multiplier} AS folate,
        choline*{multiplier} AS choline,
        vit_b12*{multiplier} AS vit_b12,
        vit_c*{multiplier} AS vit_c,
        vit_d*{multiplier} AS vit_d,
        vit_k*{multiplier} AS vit_k,
        calcium*{multiplier} AS calcium,
        phosphorus*{multiplier} AS phosphorus,
        magnesium*{multiplier} AS magnesium,
        iron*{multiplier} AS iron,
        zinc*{multiplier} AS zinc,
        sodium*{multiplier} AS sodium,
        potassium*{multiplier} AS potassium,
        POWER(({multiplier}*vit_e - {vit_e}), 2) as vit_e_2, 
        POWER(({multiplier}*vit_a - {vit_a}), 2) as vit_a_2, 
        POWER(({multiplier}*thiamin_b1 - {thiamin_b1}), 2) as thiamin_b1_2, 
        POWER(({multiplier}*riboflavin_b2 - {riboflavin_b2}), 2) as riboflavin_b2_2, 
        POWER(({multiplier}*niacin - {niacin}), 2) as niacin_2, 
        POWER(({multiplier}*vit_b6 - {vit_b6}), 2) as vit_b6_2, 
        POWER(({multiplier}*folate - {folate}), 2) as folate_2, 
        POWER(({multiplier}*choline - {choline}), 2) as choline_2, 
        POWER(({multiplier}*vit_b12 - {vit_b12}), 2) as vit_b12_2, 
        POWER(({multiplier}*vit_c - {vit_c}), 2) as vit_c_2, 
        POWER(({multiplier}*vit_d - {vit_d}), 2) as vit_d_2, 
        POWER(({multiplier}*vit_k - {vit_k}), 2) as vit_k_2, 
        POWER(({multiplier}*calcium - {calcium}), 2) as calcium_2, 
        POWER(({multiplier}*phosphorus - {phosphorus}), 2) as phosphorus_2, 
        POWER(({multiplier}*magnesium - {magnesium}), 2) as magnesium_2, 
        POWER(({multiplier}*iron - {iron}), 2) as iron_2, 
        POWER(({multiplier}*zinc - {zinc}), 2) as zinc_2, 
        POWER(({multiplier}*sodium - {sodium}), 2) as sodium_2, 
        POWER(({multiplier}*potassium - {potassium}), 2) as potassium_2,
        {multiplier}*100.0 as serving
    FROM food)

    SELECT fdcid, description, serving, sqrt((vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
        vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2)/19) AS rmse,
        vit_e,
        vit_a,
        thiamin_b1,
        riboflavin_b2,
        niacin,
        vit_b6,
        folate,
        choline,
        vit_b12,
        vit_c,
        vit_d,
        vit_k,
        calcium,
        phosphorus,
        magnesium,
        iron,
        zinc,
        sodium,
        potassium
    FROM errors
    ORDER BY rmse ASC
    LIMIT 100
    '''

    foods = pd.read_sql(myquery, con=engine)

    multipliers = np.arange(.2, 4, 0.1).tolist()
    for multiplier in multipliers:
        myquery = f'''
        WITH errors AS (SELECT fdcid, description,
            vit_e*{multiplier} AS vit_e,
            vit_a*{multiplier} AS vit_a,
            thiamin_b1*{multiplier} AS thiamin_b1,
            riboflavin_b2*{multiplier} AS riboflavin_b2,
            niacin*{multiplier} AS niacin,
            vit_b6*{multiplier} AS vit_b6,
            folate*{multiplier} AS folate,
            choline*{multiplier} AS choline,
            vit_b12*{multiplier} AS vit_b12,
            vit_c*{multiplier} AS vit_c,
            vit_d*{multiplier} AS vit_d,
            vit_k*{multiplier} AS vit_k,
            calcium*{multiplier} AS calcium,
            phosphorus*{multiplier} AS phosphorus,
            magnesium*{multiplier} AS magnesium,
            iron*{multiplier} AS iron,
            zinc*{multiplier} AS zinc,
            sodium*{multiplier} AS sodium,
            potassium*{multiplier} AS potassium,
            POWER(({multiplier}*vit_e - {vit_e}), 2) as vit_e_2, 
            POWER(({multiplier}*vit_a - {vit_a}), 2) as vit_a_2, 
            POWER(({multiplier}*thiamin_b1 - {thiamin_b1}), 2) as thiamin_b1_2, 
            POWER(({multiplier}*riboflavin_b2 - {riboflavin_b2}), 2) as riboflavin_b2_2, 
            POWER(({multiplier}*niacin - {niacin}), 2) as niacin_2, 
            POWER(({multiplier}*vit_b6 - {vit_b6}), 2) as vit_b6_2, 
            POWER(({multiplier}*folate - {folate}), 2) as folate_2, 
            POWER(({multiplier}*choline - {choline}), 2) as choline_2, 
            POWER(({multiplier}*vit_b12 - {vit_b12}), 2) as vit_b12_2, 
            POWER(({multiplier}*vit_c - {vit_c}), 2) as vit_c_2, 
            POWER(({multiplier}*vit_d - {vit_d}), 2) as vit_d_2, 
            POWER(({multiplier}*vit_k - {vit_k}), 2) as vit_k_2, 
            POWER(({multiplier}*calcium - {calcium}), 2) as calcium_2, 
            POWER(({multiplier}*phosphorus - {phosphorus}), 2) as phosphorus_2, 
            POWER(({multiplier}*magnesium - {magnesium}), 2) as magnesium_2, 
            POWER(({multiplier}*iron - {iron}), 2) as iron_2, 
            POWER(({multiplier}*zinc - {zinc}), 2) as zinc_2, 
            POWER(({multiplier}*sodium - {sodium}), 2) as sodium_2, 
            POWER(({multiplier}*potassium - {potassium}), 2) as potassium_2,
            {multiplier}*100.0 as serving
        FROM food)

        SELECT fdcid, description, serving, SQRT((vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
            vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2)/19) AS rmse,
            vit_e,
            vit_a,
            thiamin_b1,
            riboflavin_b2,
            niacin,
            vit_b6,
            folate,
            choline,
            vit_b12,
            vit_c,
            vit_d,
            vit_k,
            calcium,
            phosphorus,
            magnesium,
            iron,
            zinc,
            sodium,
            potassium
        FROM errors
        ORDER BY rmse ASC
        LIMIT 100
        '''

        new_foods = pd.read_sql(myquery, con=engine)
        foods = pd.concat([foods, new_foods], axis=0)
        

    # graph the thing
    print("This graph is based on " + str(mean_nutr['n'][0]) + " people")
    return foodgraph(foods)

In [32]:
myGraph = make_nutrition_graph(55, 'F', race = None, education = None, marital = None, poverty = 'Yes')

This graph is based on 307 people


In [33]:
demo.head()

,seqn,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,College graduate or above,Never married,5.00
4,109269.0,M,2.0,Other Hispanic,NaN,NaN,0.96


In [34]:
demo['age_year'].max()

80.0

In [35]:
external_stylesheets = [dbc.themes.LITERA]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [36]:
app.layout = html.Div(
[
    html.H1("Nutrition Recommendation Dashboard"),
    dcc.Markdown("Age"),
    dcc.Slider(id="ageSlider", min = 2, max = 80, step = 1, value = 30),
    html.Div(
        [
        dcc.Markdown("Sex"),
        dcc.Dropdown(id="sexDropdown", options=list(demo['sex'].unique()), value = 'F'),
        dcc.Markdown("Race"),
        dcc.Dropdown(id="raceDropdown", options=list(demo['race'].unique()), value = None),
        dcc.Markdown("Education Level"),
        dcc.Dropdown(id="educDropdown", options=list(demo["education"].dropna().unique()), value = None),
        dcc.Markdown("Marital Status"),
        dcc.Dropdown(id="maritalDropdown", options=list(demo["marital_status"].dropna().unique()), value = None),
        dcc.Markdown("Poverty Status"),
        dcc.Dropdown(id="povertyDropdown", options=list(demo['income_pov_rat'].unique()), value = None)],
        style = {"width":"20%", "float":"left"}),
    
    html.Div(
        [dcc.Graph(id="nutritionGraph")],
        style = {"width":"75%", "float":"right"}),
    
]


)

In [37]:
[x for x in list(demo['education'].unique()) if not x == np.nan]

[nan,
 'College graduate or above',
 '9-11th grade',
 'Some college or AA degree',
 'High school graduate/GED or equivalent',
 'Less than 9th grade']

In [38]:
list(demo["education"].dropna().unique())

['College graduate or above',
 '9-11th grade',
 'Some college or AA degree',
 'High school graduate/GED or equivalent',
 'Less than 9th grade']

In [39]:
@app.callback(Output(component_id = "nutritionGraph", component_property = "figure"), 
             [Input(component_id = "ageSlider", component_property = "value"),
             Input(component_id = "sexDropdown", component_property = "value"),
             Input(component_id = "raceDropdown", component_property = "value"),
             Input(component_id = "educDropdown", component_property = "value"),
             Input(component_id = "maritalDropdown", component_property = "value"),
             Input(component_id = "povertyDropdown", component_property = "value")])

def make_nutrition_graph(age, sex, race = None, education = None, marital = None, poverty = None):
    
    # get the string to search the demo and population nutrition tables
    
    strings = get_search_string(age, sex, race, education, marital, poverty)
    dem_string = strings[0]
    nutr_string = strings[1]

    # search the demo and population nutrition tables
    
    mean_nutr_query = f'''
    SELECT COUNT(d.age_year) as n, AVG(avg_vit_e) AS vit_e, AVG(avg_vit_a) AS vit_a, AVG(avg_thiamin_b1) AS thiamin_b1, AVG(avg_riboflavin_b2) AS riboflavin_b2, 
        AVG(avg_niacin) AS niacin, AVG(avg_vit_b6) AS vit_b6, AVG(avg_folate) AS folate, AVG(avg_choline) AS choline, AVG(avg_vit_b12) AS vit_b12, 
        AVG(avg_vit_c) AS vit_c, AVG(avg_vit_d) AS vit_d, AVG(avg_vit_k) AS vit_k, AVG(avg_calcium) AS calcium, AVG(avg_phosphorus) AS phosphorus, 
        AVG(avg_magnesium) AS magnesium, AVG(avg_iron) AS iron, AVG(avg_zinc) AS zinc, AVG(avg_sodium) AS sodium, AVG(avg_potassium) AS potassium
    FROM demo d 
    INNER JOIN pop_nutr p
        ON d.seqn = p.seqn
    WHERE {dem_string}
    GROUP BY sex
    '''

    mean_nutr = pd.read_sql(mean_nutr_query, con=engine)
    
    myquery = f'''
    SELECT vit_e, vit_a, thiamin_b1, riboflavin_b2, niacin, vit_b6, folate, choline, vit_b12, vit_c, vit_d, vit_k, calcium, phosphorus, 
        magnesium, iron, zinc, sodium, potassium
    FROM rec
    WHERE {nutr_string}
    '''

    recommendation = pd.read_sql(myquery, con=engine)

    # subtract the searches to get the deficit and store those deficit values
    
    needs = recommendation.subtract(mean_nutr.drop(['n'], axis = 1))
    vit_e = needs['vit_e'][0]
    vit_a = needs['vit_a'][0]
    thiamin_b1 = needs['thiamin_b1'][0]
    riboflavin_b2 = needs['riboflavin_b2'][0]
    niacin = needs['niacin'][0]
    vit_b6 = needs['vit_b6'][0]
    folate = needs['folate'][0]
    choline = needs['choline'][0]
    vit_b12 = needs['vit_b12'][0]
    vit_c = needs['vit_c'][0]
    vit_d = needs['vit_d'][0]
    vit_k = needs['vit_k'][0]
    calcium = needs['calcium'][0]
    phosphorus = needs['phosphorus'][0]
    magnesium = needs['magnesium'][0]
    iron = needs['iron'][0]
    zinc = needs['zinc'][0]
    sodium = needs['sodium'][0]
    potassium = needs['potassium'][0]

    # calculate least squares for foods and order them smallest to largest
    
    multiplier = .1
    myquery = f'''
    WITH errors AS (SELECT fdcid, description,
        vit_e*{multiplier} AS vit_e,
        vit_a*{multiplier} AS vit_a,
        thiamin_b1*{multiplier} AS thiamin_b1,
        riboflavin_b2*{multiplier} AS riboflavin_b2,
        niacin*{multiplier} AS niacin,
        vit_b6*{multiplier} AS vit_b6,
        folate*{multiplier} AS folate,
        choline*{multiplier} AS choline,
        vit_b12*{multiplier} AS vit_b12,
        vit_c*{multiplier} AS vit_c,
        vit_d*{multiplier} AS vit_d,
        vit_k*{multiplier} AS vit_k,
        calcium*{multiplier} AS calcium,
        phosphorus*{multiplier} AS phosphorus,
        magnesium*{multiplier} AS magnesium,
        iron*{multiplier} AS iron,
        zinc*{multiplier} AS zinc,
        sodium*{multiplier} AS sodium,
        potassium*{multiplier} AS potassium,
        POWER(({multiplier}*vit_e - {vit_e}), 2) as vit_e_2, 
        POWER(({multiplier}*vit_a - {vit_a}), 2) as vit_a_2, 
        POWER(({multiplier}*thiamin_b1 - {thiamin_b1}), 2) as thiamin_b1_2, 
        POWER(({multiplier}*riboflavin_b2 - {riboflavin_b2}), 2) as riboflavin_b2_2, 
        POWER(({multiplier}*niacin - {niacin}), 2) as niacin_2, 
        POWER(({multiplier}*vit_b6 - {vit_b6}), 2) as vit_b6_2, 
        POWER(({multiplier}*folate - {folate}), 2) as folate_2, 
        POWER(({multiplier}*choline - {choline}), 2) as choline_2, 
        POWER(({multiplier}*vit_b12 - {vit_b12}), 2) as vit_b12_2, 
        POWER(({multiplier}*vit_c - {vit_c}), 2) as vit_c_2, 
        POWER(({multiplier}*vit_d - {vit_d}), 2) as vit_d_2, 
        POWER(({multiplier}*vit_k - {vit_k}), 2) as vit_k_2, 
        POWER(({multiplier}*calcium - {calcium}), 2) as calcium_2, 
        POWER(({multiplier}*phosphorus - {phosphorus}), 2) as phosphorus_2, 
        POWER(({multiplier}*magnesium - {magnesium}), 2) as magnesium_2, 
        POWER(({multiplier}*iron - {iron}), 2) as iron_2, 
        POWER(({multiplier}*zinc - {zinc}), 2) as zinc_2, 
        POWER(({multiplier}*sodium - {sodium}), 2) as sodium_2, 
        POWER(({multiplier}*potassium - {potassium}), 2) as potassium_2,
        {multiplier}*100.0 as serving
    FROM food)

    SELECT fdcid, description, serving, sqrt((vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
        vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2)/19) AS rmse,
        vit_e,
        vit_a,
        thiamin_b1,
        riboflavin_b2,
        niacin,
        vit_b6,
        folate,
        choline,
        vit_b12,
        vit_c,
        vit_d,
        vit_k,
        calcium,
        phosphorus,
        magnesium,
        iron,
        zinc,
        sodium,
        potassium
    FROM errors
    ORDER BY rmse ASC
    LIMIT 100
    '''

    foods = pd.read_sql(myquery, con=engine)

    multipliers = np.arange(.2, 4, 0.1).tolist()
    for multiplier in multipliers:
        myquery = f'''
        WITH errors AS (SELECT fdcid, description,
            vit_e*{multiplier} AS vit_e,
            vit_a*{multiplier} AS vit_a,
            thiamin_b1*{multiplier} AS thiamin_b1,
            riboflavin_b2*{multiplier} AS riboflavin_b2,
            niacin*{multiplier} AS niacin,
            vit_b6*{multiplier} AS vit_b6,
            folate*{multiplier} AS folate,
            choline*{multiplier} AS choline,
            vit_b12*{multiplier} AS vit_b12,
            vit_c*{multiplier} AS vit_c,
            vit_d*{multiplier} AS vit_d,
            vit_k*{multiplier} AS vit_k,
            calcium*{multiplier} AS calcium,
            phosphorus*{multiplier} AS phosphorus,
            magnesium*{multiplier} AS magnesium,
            iron*{multiplier} AS iron,
            zinc*{multiplier} AS zinc,
            sodium*{multiplier} AS sodium,
            potassium*{multiplier} AS potassium,
            POWER(({multiplier}*vit_e - {vit_e}), 2) as vit_e_2, 
            POWER(({multiplier}*vit_a - {vit_a}), 2) as vit_a_2, 
            POWER(({multiplier}*thiamin_b1 - {thiamin_b1}), 2) as thiamin_b1_2, 
            POWER(({multiplier}*riboflavin_b2 - {riboflavin_b2}), 2) as riboflavin_b2_2, 
            POWER(({multiplier}*niacin - {niacin}), 2) as niacin_2, 
            POWER(({multiplier}*vit_b6 - {vit_b6}), 2) as vit_b6_2, 
            POWER(({multiplier}*folate - {folate}), 2) as folate_2, 
            POWER(({multiplier}*choline - {choline}), 2) as choline_2, 
            POWER(({multiplier}*vit_b12 - {vit_b12}), 2) as vit_b12_2, 
            POWER(({multiplier}*vit_c - {vit_c}), 2) as vit_c_2, 
            POWER(({multiplier}*vit_d - {vit_d}), 2) as vit_d_2, 
            POWER(({multiplier}*vit_k - {vit_k}), 2) as vit_k_2, 
            POWER(({multiplier}*calcium - {calcium}), 2) as calcium_2, 
            POWER(({multiplier}*phosphorus - {phosphorus}), 2) as phosphorus_2, 
            POWER(({multiplier}*magnesium - {magnesium}), 2) as magnesium_2, 
            POWER(({multiplier}*iron - {iron}), 2) as iron_2, 
            POWER(({multiplier}*zinc - {zinc}), 2) as zinc_2, 
            POWER(({multiplier}*sodium - {sodium}), 2) as sodium_2, 
            POWER(({multiplier}*potassium - {potassium}), 2) as potassium_2,
            {multiplier}*100.0 as serving
        FROM food)

        SELECT fdcid, description, serving, SQRT((vit_e_2 + vit_a_2 + thiamin_b1_2 + riboflavin_b2_2 + niacin_2 + vit_b6_2 + folate_2 + choline_2 + vit_b12_2 + vit_c_2 + 
            vit_d_2 + vit_k_2 + calcium_2 + phosphorus_2 + magnesium_2 + iron_2 + zinc_2 + sodium_2 + potassium_2)/19) AS rmse,
            vit_e,
            vit_a,
            thiamin_b1,
            riboflavin_b2,
            niacin,
            vit_b6,
            folate,
            choline,
            vit_b12,
            vit_c,
            vit_d,
            vit_k,
            calcium,
            phosphorus,
            magnesium,
            iron,
            zinc,
            sodium,
            potassium
        FROM errors
        ORDER BY rmse ASC
        LIMIT 100
        '''

        new_foods = pd.read_sql(myquery, con=engine)
        foods = pd.concat([foods, new_foods], axis=0)
        

    # graph the thing
    print("This graph is based on " + str(mean_nutr['n'][0]) + " people")
    return foodgraph(foods)

In [40]:
if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

Dash app running on http://0.0.0.0:8050/
